# Camera Object Detection & Distance Estimation
This notebook displays the JetCam camera feed, detects the main object, draws a bounding box, and estimates the distance using a single reference point.

In [1]:
!echo $USER | sudo -S systemctl restart nvargus-daemon 
%cd fast-and-furious-with-self-drive-ai/scripts/helpers/capture_reference_with_distance.ipynb

[sudo] password for jetson: 

[Errno 20] Not a directory: 'fast-and-furious-with-self-drive-ai/scripts/helpers/capture_reference_with_distance.ipynb'
/home/jetson


In [2]:
import cv2
import numpy as np
from jetcam.csi_camera import CSICamera
from IPython.display import display, clear_output
import ipywidgets as widgets

In [3]:
# Reference calibration (update as needed)
REFERENCE_AREA = 43456  # pixels
REFERENCE_DISTANCE = 0.20  # meters

In [4]:
# Initialize camera
camera = CSICamera(width=224, height=224, capture_fps=30)
camera.running = True

In [5]:
# Helper: Show HSV value at mouse click
import matplotlib.pyplot as plt

def show_hsv(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title("Click on the bottle to get HSV value")
    coords = plt.ginput(1)
    if coords:
        x, y = int(coords[0][0]), int(coords[0][1])
        print("HSV at ({}, {}): {}".format(x, y, hsv[y, x]))
    plt.show()

# Usage: Run this cell, then call show_hsv(camera.value.copy()) and click on the bottle in the image.

In [6]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
box_widget = widgets.Image(format='jpeg', width=224, height=224)

In [7]:
import collections
bbox_history = collections.deque(maxlen=5)  # For smoothing bounding box
def process_frame(frame):
    # Convert to HSV for color segmentation
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Blur to reduce noise
    hsv = cv2.GaussianBlur(hsv, (7,7), 0)
    # Define red/maroon color range (tune as needed)
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    # Combine masks for red hue range
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)
    # Morphological operations to clean up mask
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((7,7), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    annotated = frame.copy()
    distance = None
    min_area = 500  # Ignore small contours
    if contours:
        largest = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest) > min_area:
            x, y, w, h = cv2.boundingRect(largest)
            bbox_history.append((x, y, w, h))
            # Smooth bounding box
            if len(bbox_history) > 1:
                x = int(np.mean([b[0] for b in bbox_history]))
                y = int(np.mean([b[1] for b in bbox_history]))
                w = int(np.mean([b[2] for b in bbox_history]))
                h = int(np.mean([b[3] for b in bbox_history]))
            area = w * h
            if area > 0:
                distance = REFERENCE_DISTANCE * np.sqrt(REFERENCE_AREA / area)
                cv2.rectangle(annotated, (x, y), (x+w, y+h), (0,255,0), 2)
                cv2.putText(annotated, f'Dist: {distance:.2f}m', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    return annotated

In [8]:
import threading
import time
def update():
    while camera.running:
        frame = camera.value.copy() if hasattr(camera, 'value') else camera.read()
        annotated = process_frame(frame)
        _, jpeg = cv2.imencode('.jpg', annotated)
        box_widget.value = jpeg.tobytes()
        time.sleep(0.1)  # ~10 FPS

In [9]:
thread = threading.Thread(target=update)
thread.daemon = True
thread.start()
display(box_widget)

Image(value=b'', format='jpeg', height='224', width='224')